In [1]:
%matplotlib inline
import os
from tomato.audio.AudioAnalyzer import AudioAnalyzer
from tomato.symbolic.SymbTrAnalyzer import SymbTrAnalyzer
from tomato.joint.JointAnalyzer import JointAnalyzer
import pylab
pylab.rcParams['figure.figsize'] = (20, 8)


JointAnalyzer assumes the individual audio analysis and score analysis is applied earlier. 

In [2]:
# instantiate
audio_filepath = os.path.join('ussak--sazsemaisi--aksaksemai----neyzen_aziz_dede',
                              'f970f1e0-0be9-4914-8302-709a0eac088e',
                              'f970f1e0-0be9-4914-8302-709a0eac088e.mp3')
audio_feature_filepath = os.path.join('ussak--sazsemaisi--aksaksemai----neyzen_aziz_dede',
                                      'f970f1e0-0be9-4914-8302-709a0eac088e',
                                      'f970f1e0-0be9-4914-8302-709a0eac088e.pkl')
audio_features = AudioAnalyzer.from_pickle(audio_feature_filepath)

# score filepaths
score_filepath = os.path.join('ussak--sazsemaisi--aksaksemai----neyzen_aziz_dede',
                              'ussak--sazsemaisi--aksaksemai----neyzen_aziz_dede.txt')
score_feature_filepath = os.path.join('ussak--sazsemaisi--aksaksemai----neyzen_aziz_dede',
                                      'ussak--sazsemaisi--aksaksemai----neyzen_aziz_dede.pkl')
score_features = SymbTrAnalyzer.from_pickle(score_feature_filepath)

# instantiate analyzer object
jointAnalyzer = JointAnalyzer(verbose=True)
